<a href="https://colab.research.google.com/github/Squatsit/CS6405/blob/main/Model_for_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
df = pd.read_csv("https://github.com/andvise/DataAnalyticsDatasets/blob/main/test_dataset.csv?raw=True",index_col=0)
old_shape=df.shape


df=df.dropna(axis=1) 
#originally tried rows but all were removed



#replace with na and then drop these rows
df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=1) 


df_target=df['target']
df=df.drop(columns=['target'])

from sklearn import model_selection
train_features,test_features,train_labels,test_labels=model_selection.train_test_split(df,df_target,test_size=0.3, random_state=17)

from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_labels=le.fit_transform(train_labels)
test_labels=le.transform(test_labels)

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import VarianceThreshold


#Remove all features which have a variance below 0.03
constant_filter = VarianceThreshold(threshold=0.04)#0.03 - 0.9733 0.04 - 0.979
train_features0=constant_filter.fit_transform(train_features)
test_features0=constant_filter.transform(test_features)

r,c=train_features0.shape

FS_classifierRF = Pipeline([
    ("feature_selection",SelectKBest(f_classif)),
    ("predictor", RandomForestClassifier())
    ])

param_grid = {"feature_selection__k":['all'],
              "predictor__n_estimators": [200],
              "predictor__criterion": ["entropy"],
              "predictor__max_features":[None]
              }
              
#cross validation
FS_RF_gs = GridSearchCV(FS_classifierRF, param_grid, scoring="accuracy",cv=10)

# Run the GridSearchCV
FS_RF_gs.fit(train_features0, train_labels)

# Print the best parameters and the score
FS_RF_gs.best_params_, FS_RF_gs.best_score_

from joblib import dump
dump(FS_classifierRF,'./bestmodel.joblib')

['./bestmodel.joblib']